In [1]:
import os
import sys

In [2]:
os.chdir('/home/au643300/DataHandling/')
sys.path.append('/home/au643300/DataHandling/')
os.environ['HDF5_USE_FILE_LOCKING']="FALSE"

In [3]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, as_completed,wait,fire_and_forget,progress
import glob
from src.data.tonetCDF import to_netcdf
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from src.interim import read_valdata
import numpy as np
import importlib
import dask
import zarr
import time
from rechunker import rechunk

In [4]:
sns.set_theme()
dask.config.set({"distributed.comm.timeouts.tcp": "50s"})
dask.config.set({'distributed.comm.retry.count':3})
dask.config.set({'distributed.comm.timeouts.connect':'25s'})
dask.config.set({"distributed.worker.use-file-locking":False})
zarr.blosc.use_threads = False

In [5]:
cluster=SLURMCluster(cores=8,
                     memory="50GB",
                     queue='q64',
                     walltime='0-06:00:00',
                     local_directory='/scratch/$SLURM_JOB_ID',
                     interface='ib0',
                     scheduler_options={'interface':'ib0'},
                     extra=['--resources mem=15',"--lifetime", "340m", "--lifetime-stagger", "4m"]
                    )
                    

In [6]:
client=Client(cluster)

In [7]:
cluster.adapt(minimum=1,maximum=3)

In [8]:
client

Client Scheduler: tcp://10.100.8.23:37438 Dashboard: http://10.100.8.23:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


Now trying to rechunk

In [9]:
source=xr.open_zarr("/home/au643300/NOBACKUP/data.zarr",consolidated=True)
source

,Array,Chunk
Bytes,71.77 GB,105.38 MB
Shape,"(681, 256, 201, 256)","(1, 256, 201, 256)"
Count,682 Tasks,681 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,71.77 GB,105.38 MB
Shape,"(681, 256, 201, 256)","(1, 256, 201, 256)"
Count,682 Tasks,681 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [10]:
temp="/home/au643300/NOBACKUP/rechunk/temp.zarr"
target ="/home/au643300/NOBACKUP/rechunk/data.zarr"
max_mem=40000000000
compressor = zarr.Blosc(cname="zstd", clevel=4, shuffle=2)
chunks_dict={"pr0.025": {'time':4,'x': 256,'y': 201,'z': 256},
             "pr0.2": {'time':4,'x': 256,'y': 201,'z': 256},
             "pr1": {'time':4,'x': 256,'y': 201,'z': 256},
             "time" : None,
             "u_vel": {'time':4,'x': 256,'y': 201,'z': 256},
             "v_vel": {'time':4,'x': 256,'y': 201,'z': 256},
             "w_vel": {'time':4,'x': 256,'y': 201,'z': 256},
             "x": None,
             "y": None,
             "z": None,    
            }
compression_dict={"pr0.025": {"compressor": compressor},
             "pr0.2": {"compressor": compressor},
             "pr1": {"compressor": compressor},
             "time" : {"compressor": compressor},
             "u_vel": {"compressor": compressor},
             "v_vel": {"compressor": compressor},
             "w_vel": {"compressor": compressor},
             "x": {"compressor": compressor},
             "y": {"compressor": compressor},
             "z": {"compressor": compressor},  
            }

In [12]:
array_plan=rechunk(source,chunks_dict,max_mem,target,temp_store=temp,target_options=compression_dict)


In [ ]:
future=array_plan.execute(retries=25,inline_array=True)
